In [19]:
import gradio as gr
import lancedb
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader
import os
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import pandas as pd
from llama_index.core import ServiceContext

In [20]:
# Settings를 사용하여 임베딩 및 LLM 설정
#Settings.embed_model = HuggingFaceEmbedding(model_name="bert-base-uncased")  # 768 차원의 벡터를 생성하는 모델
#Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

#all-mpnet-base-v2 (768차원, 정확도 높음, 속도 중간)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")


#Settings.embed_model = HuggingFaceEmbedding(model_name="gemma2")

# 📌 Ollama 모델 설정 (로컬 모델 사용)
#llm = OllamaLLM(model_name="gemma2")  # Ollama 모델 설정
#Settings.llm = llm


llm = Ollama(model="gemma2",base_url="http://localhost:11434/v1")
Settings.llm = llm

In [21]:
# 📌 GraphRAG에서 생성한 LanceDB 불러오기
db_path = "/Users/jun/GitStudy/Data_4/Data/project5/model/graphrag_t_2/output/lancedb"
db = lancedb.connect(db_path)  # GraphRAG에서 사용한 DB 경로 
db

LanceDBConnection(/Users/jun/GitStudy/Data_4/Data/project5/model/graphrag_t_2/output/lancedb)

In [37]:
db.open_table("default-community-full_content").to_pandas()

,id,text,vector,attributes
0,0985dfb0576a493a90740467e97a46fd,# ADVANCED SNAIL 96 MUCIN POWER ESSENCE and RE...,"[0.017612401, 0.0761737, -0.17061758, -0.07786...","{""title"": ""# ADVANCED SNAIL 96 MUCIN POWER ESS..."


In [22]:
# ✅ 테이블 존재 여부 확인 후 불러오기
table_names = db.table_names()
# ✅ 모든 테이블을 불러와서 VectorStoreIndex 생성
vector_stores = {name: LanceDBVectorStore(table=db.open_table(name)) for name in table_names}
vector_stores

{'default-community-full_content': LanceDBVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, uri='/tmp/lancedb', vector_column_name='vector', nprobes=20, refine_factor=None, text_key='text', doc_id_key='doc_id', api_key=None, region=None, mode='overwrite', query_type='vector', overfetch_factor=1),
 'default-entity-description': LanceDBVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, uri='/tmp/lancedb', vector_column_name='vector', nprobes=20, refine_factor=None, text_key='text', doc_id_key='doc_id', api_key=None, region=None, mode='overwrite', query_type='vector', overfetch_factor=1),
 'default-text_unit-text': LanceDBVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, uri='/tmp/lancedb', vector_column_name='vector', nprobes=20, refine_factor=None, text_key='text', doc_id_key='doc_id', api_key=None, region=None, mode='overwrite', query_type='vector', overfetch_factor=1)}

In [7]:
storage_contexts = {name: StorageContext.from_defaults(vector_store=store) for name, store in vector_stores.items()}
storage_contexts

{'default-community-full_content': StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x316b36750>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x344105010>, vector_stores={'default': LanceDBVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, uri='/tmp/lancedb', vector_column_name='vector', nprobes=20, refine_factor=None, text_key='text', doc_id_key='doc_id', api_key=None, region=None, mode='overwrite', query_type='vector', overfetch_factor=1), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x3440fbdd0>, property_graph_store=None),
 'default-entity-description': StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x3

In [8]:
indexes = {name: VectorStoreIndex.from_vector_store(store, storage_context=storage_contexts[name]) for name, store in vector_stores.items()}
indexes

{'default-community-full_content': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x34410d990>,
 'default-entity-description': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x3440cee50>,
 'default-text_unit-text': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x3440cd310>}

In [9]:
# ✅ 여러 테이블을 사용할 수 있도록 Query 엔진을 딕셔너리 형태로 저장
query_engines = {name: index.as_query_engine() for name, index in indexes.items()}
query_engines

{'default-community-full_content': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x341f87210>,
 'default-entity-description': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x3440c5890>,
 'default-text_unit-text': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x3440c56d0>}

In [14]:
# 📌 파일 업로드 후 문서 처리
def process_uploaded_files(files):
    """사용자가 업로드한 파일을 처리하여 LlamaIndex에 추가"""
    if not files:
        return None  # 파일이 없으면 무시

    # 파일 저장 경로
    upload_dir = "uploaded_files"
    os.makedirs(upload_dir, exist_ok=True)

    # 업로드된 파일 저장
    file_paths = []
    for file in files:
        file_path = os.path.join(upload_dir, file.name)
        file_paths.append(file_path)

        # Parquet 파일 처리
        if file.name.endswith(".parquet"):
            try:
                df = pd.read_parquet(file)  # ✅ Parquet 파일을 DataFrame으로 변환
                text_data = df.to_string(index=False)  # ✅ DataFrame을 문자열로 변환
                text_file_path = file_path.replace(".parquet", ".txt")  # 변환된 파일명
                with open(text_file_path, "w", encoding="utf-8") as text_file:
                    text_file.write(text_data)  # ✅ TXT 파일로 저장하여 LlamaIndex가 읽을 수 있도록 변환
                file_paths.append(text_file_path)  # 변환된 파일을 추가
            except Exception as e:
                print(f"❌ Parquet 파일 변환 실패: {e}")
                return None  # 변환 실패 시 무시

        else:
            # 기존 방식대로 파일 저장
            with open(file_path, "wb") as f:
                f.write(file.read())

    # 📌 새 문서 인덱싱
    documents = SimpleDirectoryReader(input_files=file_paths).load_data()
    new_index = VectorStoreIndex.from_documents(documents)
    return new_index.as_query_engine()

# 📌 사용자 메시지 처리
def answer(message, history, files):
    
    global query_engines
    """사용자의 질문을 받고, 기존 GraphRAG 데이터 + 업로드된 문서 데이터로 답변"""
    
    # 기존 GraphRAG 데이터 (여러 테이블에서 처리)
    query_engine_list = list(query_engines.values())  
    
    # 업로드된 파일이 있을 경우 새롭게 인덱싱하여 추가
    new_query_engine = process_uploaded_files(files)
    if new_query_engine:
        query_engine_list.append(new_query_engine)
        query_engines["uploaded_files"] = new_query_engine # ✅ 업로드한 문서도 추가
    
    # 모든 쿼리 엔진에서 질의 수행
    responses = []
    for qe in query_engine_list:
        print(qe)
        if hasattr(qe, 'query'):  # query 엔진이 query 메서드를 가지고 있는지 확인
            responses.append(qe.query(message["text"]))
        else:
            print(f"❌ {qe}는 query 메서드를 가지고 있지 않습니다.")
    
    # 📌 응답을 종합하여 반환
    return "\n\n---\n\n".join([str(resp) for resp in responses])

In [25]:
# 📌 Gradio 인터페이스 설정
demo = gr.ChatInterface(
    answer,
    type="messages",
    title="GraphRAG + Ollama RAG Chatbot",
    description="GraphRAG에서 생성한 LanceDB 데이터와 사용자가 업로드한 문서를 활용한 Ollama 기반 RAG Chatbot!",
    textbox=gr.MultimodalTextbox(file_types=[".pdf", ".txt"]),
    multimodal=True  # 파일 업로드 허용
)

# 📌 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


/opt/homebrew/Caskroom/miniconda/base/envs/leo4study/lib/python3.11/site-packages/gradio/helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


relation :  {}


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/leo4study/lib/python3.11/site-packages/llama_index/vector_stores/lancedb/base.py", line 529, in query
    print(f"item_json : {item_json}")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/leo4study/lib/python3.11/site-packages/llama_index/core/vector_stores/utils.py", line 70, in metadata_dict_to_node
    raise ValueError("Node content not found in metadata dict.")
ValueError: Node content not found in metadata dict.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/leo4study/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._lib